In [1]:
!pip install LibRecommender

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import numpy as np
import pandas as pd
from libreco.data import random_split, DatasetPure
from libreco.algorithms import LightGCN  # pure data, algorithm LightGCN
from libreco.evaluation import evaluate

Instructions for updating:
non-resource variables are not supported in the long term


C:\Users\user\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [3]:
products = pd.read_csv('../data/product_details.tsv', sep='\t')
users = pd.read_csv('../data/user_game_time.tsv', sep='\t')

In [4]:
data = users.merge(products, left_on='game_id', right_on='id', how='left')
data = data[['user_id', 'id', 'genre_1_id', 'game_time']].dropna()
data = data.rename(columns={'user_id': 'user', 'id': 'item', 'genre_1_id': 'label', 'game_time': 'time'})

In [5]:
data

,user,item,label,time
112,9,5.0,61.0,443
1077,98,5.0,61.0,61
1606,151,8.0,57.0,572
1746,171,5.0,61.0,324
3518,368,8.0,57.0,0
4803,497,5.0,61.0,156
5608,602,5.0,61.0,0
5952,637,5.0,61.0,65
6567,693,8.0,57.0,243
7202,758,5.0,61.0,959


In [6]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.33)

In [7]:
len(train), len(test)

(8, 4)

In [8]:
train_data, data_info = DatasetPure.build_trainset(train)
test_data = DatasetPure.build_testset(test)

In [9]:
lightgcn = LightGCN(
    task="ranking",
    data_info=data_info,
    loss_type="bpr",
    embed_size=16,
    n_epochs=3,
    lr=1e-3,
    batch_size=2048,
    num_neg=1,
    device="cpu",
)

In [10]:
lightgcn.fit(
    train_data,
    neg_sampling=True,
    verbose=2,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)


Training start time: 2024-03-16 21:30:21


train: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.94it/s]


Epoch 1 elapsed: 0.268s
	 train_loss: 0.6727


train: 100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 54.04it/s]


Epoch 2 elapsed: 0.030s
	 train_loss: 0.6717


train: 100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 104.96it/s]

Epoch 3 elapsed: 0.022s
	 train_loss: 0.6707


In [11]:
evaluate(
    model=lightgcn,
    data=test_data,
    neg_sampling=True,
    metrics=["loss", "roc_auc", "precision", "recall", "ndcg"],
)

eval_pointwise: 100%|███████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 246.45it/s]


Detect 8 unknown interaction(s), position: [0, 1, 2, 3, 4, 5, 6, 7]


eval_listwise: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 53.90it/s]


{'loss': 0.6922103128444741,
 'roc_auc': 0.75,
 'precision': 0.2,
 'recall': 1.0,
 'ndcg': 1.0}

In [12]:
lightgcn.recommend_user(user=1077, n_rec=7)

Detect unknown user: 1077


{1077: array([8., 8., 8., 8., 8., 8., 8.])}

In [13]:
from typing import List, Tuple

def get_recommendations(user_id) -> List[Tuple[int, float]]:
    indices = lightgcn.recommend_user(user=user_id, n_rec=10, filter_consumed=True)
    return [(int(indices[user_id][i]), float(lightgcn.predict(user_id, indices[user_id][i])[0])) for i in range(len(indices[user_id]))]

In [14]:
get_recommendations(9)

Detect unknown user: 9


NameError: name 'self' is not defined